In [ ]:
# Install required packages, set backend
!pip install -q dgl         # For CPU Build
!pip install -q dgl-cu101   # For CUDA 10.1 Build

In [ ]:
# Import dependencies
import torch
import torch.nn as nn
import torch.nn.functional as F

import time
import pdb
import networkx as nx
import numpy as np

import dgl
from dgl.nn.pytorch import GATConv
from dgl.nn.pytorch.conv import gatedgraphconv
!pip install -q dgl         # For CPU Build
!pip install -q dgl-cu101   # For CUDA 10.1 Build
from dgl.data import DGLDataset
import os
import numpy as np
print(dgl.__version__)
print(torch.__version__)
print(np.__version__)

0.6.1
1.8.1+cu101
1.19.5


In [ ]:
!pip install gdown

!mkdir -p data/tsp
!cd data/tsp

# Download tsp datasets (22mb each)
!gdown --id 1tlcHok1JhOtQZOIshoGtyM5P9dZfnYbZ # tsp100_validation_seed4321.pkl
!gdown --id 1woyNI8CoDJ8hyFko4NBJ6HdF4UQA0S77 # tsp100_test_seed1234.pkl

!cd ../..


Downloading...
From: https://drive.google.com/uc?id=1tlcHok1JhOtQZOIshoGtyM5P9dZfnYbZ
To: /content/tsp100_validation_seed4321.pkl
22.0MB [00:00, 102MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1woyNI8CoDJ8hyFko4NBJ6HdF4UQA0S77
To: /content/tsp100_test_seed1234.pkl
22.0MB [00:00, 83.0MB/s]


In [ ]:
!pip install pickle5
import os
import pickle5 as pickle
import numpy as np


def check_extension(filename):
    if os.path.splitext(filename)[1] != ".pkl":
        return filename + ".pkl"
    return filename

def load_dataset(filename):

    with open(check_extension(filename), 'rb') as f:
      return pickle.load(f)

     |████████████████████████████████| 133kB 3.8MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219244 sha256=fbb401f419c79b798781bf59698df3d1562fa38008877df42deeae8fe40b72ac
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5


In [ ]:
a = load_dataset('/content/tsp100_test_seed1234')

In [ ]:
print(np.array(a[0]).shape)

(100, 2)


In [ ]:
import numpy as np
def generate(TSP_nodes, seed_gen = 0 , distribution="uniform", mode = 
np.float, map_size=(100, 100)):
  ''' generates nodes in 2D space following the specified random distribution
    Inputs:
      TSP_nodes - number of nodes (int)
      seed_gen - numpy seed (int)
      distribution - np random distribution (string)
      mode - np datatype
      map_size - 2D space (int tuple)
    Output:
      nodes - (nx2 array) 2D spatial distribution of n nodes
  '''
  np.random.seed(seed_gen)
  min_coords = min(map_size)
  if distribution == 'uniform':
    tsp_coords = np.random.uniform(0,min_coords,size = (TSP_nodes,2))
    tsp_coords = np.array(tsp_coords,dtype = mode)        
  if distribution == 'normal':
    mu = min_coords//2
    sigma = int(min_coords*0.2)
    tsp_coords = np.random.normal(mu,sigma,size = (self.num_nodes,2))
    tsp_coords = np.array(tsp_coords,dtype = mode)
  nodes = abs(tsp_coords).astype(float)
  return nodes

def calculate_edges(nodes):
  ''' generates edges between all unique pairs of nodes weighted by euclidean distance
    Input:
      nodes - (nx2 array) 2D spatial distribution of n nodes
    Output:
      src - (n-1)*n/2 length array of source nodes
      dst - (n-1)*n/2 length array of destination nodes

  '''
  num_nodes = nodes.shape[0]
  src, dst = list(), list()
  weights = np.zeros((num_nodes,num_nodes))
  for i in range(weights.shape[0]):
    for j in range(weights.shape[0]):
      weights[i,j] = np.linalg.norm(nodes[i]-nodes[j])
  neighbors = np.ones((num_nodes,num_nodes))
  np.fill_diagonal(neighbors, 2)
  node_id = np.arange(num_nodes)
  for i in range(len(nodes)-1):
    for j in range(i+1, len(nodes)):
      src.append(i)
      dst.append(j)
      eucl_dist = (np.linalg.norm(nodes[i]-nodes[j]))
  return np.array(src), np.array(dst), weights, neighbors, node_id

#nodes = generate(TSP_nodes = 50, seed_gen = 1, distribution='uniform', mode=np.int)
src, dist, weights, adj, node_id = calculate_edges(nodes)
print(nodes.shape)
print(src.shape, dist.shape)
print(weights.shape)

(50, 2)
(1225,) (1225,)
(50, 50)


In [ ]:

class EucTSPGraph(DGLDataset):
    def __init__(self):
        super().__init__(name='EucTSPGraph')

    def process(self):
        nodes_data = generate(TSP_nodes = 50, seed_gen = 1, distribution='uniform', mode=np.int)
        edges_src, edges_dst, edges_weights,adj, node_id = calculate_edges(nodes_data)

        nodes_data = torch.from_numpy(nodes_data.astype(np.float32)) 
        edges_src = torch.from_numpy(edges_src)
        edges_dst = torch.from_numpy(edges_dst)
        edges_weights = torch.from_numpy(edges_weights.astype(np.float32))
        adjacency = torch.from_numpy(adj.astype(np.int))
        node_id = torch.from_numpy(node_id)


        self.graph = dgl.graph((edges_src, edges_dst), num_nodes=nodes_data.shape[0])
        self.graph.ndata['node_coord'] = nodes_data
        self.graph.ndata['weight'] = edges_weights
        self.graph.ndata['adj'] =  adjacency
        self.graph.ndata['label'] = node_id#torch.arange(start=0, end=nodes_data.shape[0])

        # source: https://docs.dgl.ai/tutorials/blitz/6_load_data.html#sphx-glr-tutorials-blitz-6-load-data-py

        # If your dataset is a node classification dataset, you will need to assign
        # masks indicating whether a node belongs to training, validation, and test set.
        n_nodes = nodes_data.shape[0]
        n_train = int(n_nodes * 0.6)
        n_val = int(n_nodes * 0.2)
        train_mask = torch.zeros(n_nodes, dtype=torch.bool)
        val_mask = torch.zeros(n_nodes, dtype=torch.bool)
        test_mask = torch.zeros(n_nodes, dtype=torch.bool)
        train_mask[:n_train] = True
        val_mask[n_train:n_train + n_val] = True
        test_mask[n_train + n_val:] = True
        self.graph.ndata['train_mask'] = train_mask
        self.graph.ndata['val_mask'] = val_mask
        self.graph.ndata['test_mask'] = test_mask

    def __getitem__(self, i):
        return self.graph

    def __len__(self):
        return 1


In [ ]:
class BatchNormNode(nn.Module):
    def __init__(self, hidden_dim):
        super(BatchNormNode, self).__init__()
        self.batch_norm = nn.BatchNorm1d(hidden_dim, track_running_stats=False)

    def forward(self, x):
        x_trans = x.transpose(1, 2).contiguous()  # Reshape input: (batch_size, hidden_dim, num_nodes)
        x_trans_bn = self.batch_norm(x_trans)
        x_bn = x_trans_bn.transpose(1, 2).contiguous()  # Reshape to original shape
        return x_bn
class BatchNormEdge(nn.Module):
    def __init__(self, hidden_dim):
        super(BatchNormEdge, self).__init__()
        self.batch_norm = nn.BatchNorm2d(hidden_dim, track_running_stats=False)

    def forward(self, e):
        e_trans = e.transpose(1, 3).contiguous()  # Reshape input: (batch_size, num_nodes, num_nodes, hidden_dim)
        e_trans_bn = self.batch_norm(e_trans)
        e_bn = e_trans_bn.transpose(1, 3).contiguous()  # Reshape to original
        return e_bn
class NodeFeatures(nn.Module):
    def __init__(self, hidden_dim):
        super(NodeFeatures, self).__init__()
        self.U = nn.Linear(hidden_dim, hidden_dim, True)
        self.V = nn.Linear(hidden_dim, hidden_dim, True)

    def forward(self, x, edge_gate):
        Ux = self.U(x)  # B x V x H
        Vx = self.V(x)  # B x V x H
        Vx = Vx.unsqueeze(1)  # extend Vx from "B x V x H" to "B x 1 x V x H"
        gateVx = edge_gate * Vx  # B x V x V x H
        x_new = Ux + torch.sum(gateVx, dim=2) / (1e-20 + torch.sum(edge_gate, dim=2))  # B x V x H
        #sum reduction
        #x_new = Ux + torch.sum(gateVx, dim=2)  # B x V x H
        return x_new
class EdgeFeatures(nn.Module):
    def __init__(self, hidden_dim):
        super(EdgeFeatures, self).__init__()
        self.U = nn.Linear(hidden_dim, hidden_dim, True)
        self.V = nn.Linear(hidden_dim, hidden_dim, True)
        
    def forward(self, x, e):
        Ue = self.U(e)
        Vx = self.V(x)
        Wx = Vx.unsqueeze(1)  # Extend Vx from "B x V x H" to "B x V x 1 x H"
        Vx = Vx.unsqueeze(2)  # extend Vx from "B x V x H" to "B x 1 x V x H"
        e_new = Ue + Vx + Wx
        return e_new
class ResidualGatedGCNLayer(nn.Module):
    def __init__(self, hidden_dim):
        super(ResidualGatedGCNLayer, self).__init__()
        self.node_feat = NodeFeatures(hidden_dim)
        self.edge_feat = EdgeFeatures(hidden_dim)
        self.bn_node = BatchNormNode(hidden_dim)
        self.bn_edge = BatchNormEdge(hidden_dim)

    def forward(self, x, e):
        e_in = e
        x_in = x
        # Edge convolution
        e_tmp = self.edge_feat(x_in, e_in)  # B x V x V x H
        # Compute edge gates
        edge_gate = F.sigmoid(e_tmp)
        # Node convolution
        x_tmp = self.node_feat(x_in, edge_gate)
        # Batch normalization
        e_tmp = self.bn_edge(e_tmp)
        x_tmp = self.bn_node(x_tmp)
        # ReLU Activation
        e = F.relu(e_tmp)
        x = F.relu(x_tmp)
        # Residual connection
        x_new = x_in + x
        e_new = e_in + e
        return x_new, e_new
class MLP(nn.Module):
    def __init__(self, hidden_dim, output_dim, L=2):
        super(MLP, self).__init__()
        self.L = L
        U = []
        for layer in range(self.L - 1):
            U.append(nn.Linear(hidden_dim, hidden_dim, True))
        self.U = nn.ModuleList(U)
        self.V = nn.Linear(hidden_dim, output_dim, True)

    def forward(self, x):
        Ux = x
        for U_i in self.U:
            Ux = U_i(Ux)  # B x H
            Ux = F.relu(Ux)  # B x H
        y = self.V(Ux)  # B x O
        return y

In [ ]:
def loss_edges(y_pred_edges, y_edges, edge_cw):
    # Edge loss
    y = F.log_softmax(y_pred_edges, dim=3)  # B x V x V x voc_edges
    y = y.permute(0, 3, 1, 2)  # B x voc_edges x V x V
    loss_edges = nn.NLLLoss(edge_cw)(y, y_edges)
    return loss_edges
def beamsearch_tour_nodes_shortest(y_pred_edges, x_edges_values, beam_size, batch_size, num_nodes,
                                   dtypeFloat, dtypeLong, probs_type='raw', random_start=False):
    """
    Performs beamsearch procedure on edge prediction matrices and returns possible TSP tours.
    Final predicted tour is the one with the shortest tour length.
    (Standard beamsearch returns the one with the highest probability and does not take length into account.)
    Args:
        y_pred_edges: Predictions for edges (batch_size, num_nodes, num_nodes)
        x_edges_values: Input edge distance matrix (batch_size, num_nodes, num_nodes)
        beam_size: Beam size
        batch_size: Batch size
        num_nodes: Number of nodes in TSP tours
        dtypeFloat: Float data type (for GPU/CPU compatibility)
        dtypeLong: Long data type (for GPU/CPU compatibility)
        probs_type: Type of probability values being handled by beamsearch (either 'raw'/'logits'/'argmax'(TODO))
        random_start: Flag for using fixed (at node 0) vs. random starting points for beamsearch
    Returns:
        shortest_tours: TSP tours in terms of node ordering (batch_size, num_nodes)
    """
    if probs_type == 'raw':
        # Compute softmax over edge prediction matrix
        y = F.softmax(y_pred_edges, dim=3)  # B x V x V x voc_edges
        # Consider the second dimension only
        y = y[:, :, :, 1]  # B x V x V
    elif probs_type == 'logits':
        # Compute logits over edge prediction matrix
        y = F.log_softmax(y_pred_edges, dim=3)  # B x V x V x voc_edges
        # Consider the second dimension only
        y = y[:, :, :, 1]  # B x V x V
        y[y == 0] = -1e-20  # Set 0s (i.e. log(1)s) to very small negative number
    # Perform beamsearch
    beamsearch = Beamsearch(beam_size, batch_size, num_nodes, dtypeFloat, dtypeLong, probs_type, random_start)
    trans_probs = y.gather(1, beamsearch.get_current_state())
    for step in range(num_nodes - 1):
        beamsearch.advance(trans_probs)
        trans_probs = y.gather(1, beamsearch.get_current_state())
    # Initially assign shortest_tours as most probable tours i.e. standard beamsearch
    ends = torch.zeros(batch_size, 1).type(dtypeLong)
    shortest_tours = beamsearch.get_hypothesis(ends)
    # Compute current tour lengths
    shortest_lens = [1e6] * len(shortest_tours)
    for idx in range(len(shortest_tours)):
        shortest_lens[idx] = tour_nodes_to_tour_len(shortest_tours[idx].cpu().numpy(),
                                                    x_edges_values[idx].cpu().numpy())
    # Iterate over all positions in beam (except position 0 --> highest probability)
    for pos in range(1, beam_size):
        ends = pos * torch.ones(batch_size, 1).type(dtypeLong)  # New positions
        hyp_tours = beamsearch.get_hypothesis(ends)
        for idx in range(len(hyp_tours)):
            hyp_nodes = hyp_tours[idx].cpu().numpy()
            hyp_len = tour_nodes_to_tour_len(hyp_nodes, x_edges_values[idx].cpu().numpy())
            # Replace tour in shortest_tours if new length is shorter than current best
            if hyp_len < shortest_lens[idx] and is_valid_tour(hyp_nodes, num_nodes):
                shortest_tours[idx] = hyp_tours[idx]
                shortest_lens[idx] = hyp_len
    return shortest_tours

In [ ]:
class ResidualGatedGCNModel(nn.Module):
    def __init__(self, num_nodes = 50, num_layers_graph = 30, num_layers_mpl = 3, hidden_size = 300):
        super(ResidualGatedGCNModel, self).__init__()
        # Define net parameters
        self.num_nodes = num_nodes
        self.node_dim = 2 #graph.node_dim
        self.out_dim = 2 
        self.hidden_dim = hidden_size
        self.num_layers = num_layers_graph
        self.mlp_layers = num_layers_mpl
        # Node and edge embedding layers/lookups
        self.nodes_coord_embedding = nn.Linear(self.node_dim, self.hidden_dim, bias=False)
        self.edges_values_embedding = nn.Linear(1, self.hidden_dim//2, bias=False)
        self.edges_embedding = nn.Embedding(self.node_dim + 1, self.hidden_dim//2)
        # Define GCN Layers
        gcn_layers = []
        for layer in range(self.num_layers):
            gcn_layers.append(ResidualGatedGCNLayer(self.hidden_dim))
        self.gcn_layers = nn.ModuleList(gcn_layers)
        # Define MLP classifiers
        self.mlp_edges = MLP(self.hidden_dim, self.out_dim - 1, self.mlp_layers)
        # self.mlp_nodes = MLP(self.hidden_dim, self.out_dim, self.mlp_layers)

    def forward(self, x_edges, x_edges_values, x_nodes, x_nodes_coord):

        # Node and edge embedding
        x = self.nodes_coord_embedding(x_nodes_coord)  # B x V x H
        e_vals = self.edges_values_embedding(x_edges_values.unsqueeze(3))  # B x V x V x H
        e_tags = self.edges_embedding(x_edges)  # B x V x V x H
        e = torch.cat((e_vals, e_tags), dim=3)
        # GCN layers
        for layer in range(self.num_layers):
            x, e = self.gcn_layers[layer](x, e)  # B x V x H, B x V x V x H
        # MLP classifier
        y_pred_edges = self.mlp_edges(e)  
        return y_pred_edges

In [ ]:
def accuracy(logits, labels):
    indices = torch.argmax(logits, dim=1)               # indices with highest value
    num_correct = torch.sum(indices == labels)          # how many predictions match labels
    return (num_correct.item()*1.0)/len(labels)         # convert to float and find percentage 

def evaluate(model, features, labels, mask):
    model.eval()
    with torch.no_grad():                               # deactivate autograd during eval
        logits = model(features)
        bs_nodes = beamsearch_tour_nodes_shortest(y_preds, x_edges_values, beam_size, batch_size, num_nodes, dtypeFloat, dtypeLong, probs_type='logits')
        
        # Compute mean tour length
        pred_tour_len = mean_tour_len_nodes(x_edges_values, bs_nodes)
        gt_tour_len = np.mean(batch.tour_len)

        logits = logits[mask]
        labels = labels[mask]
        return accuracy(logits, labels)

def train(model, features, labels, mask):
    model.train()
"""
TO DO: incorporate labels generated by Concorde for edge mask generation
"""
    # Compute class weights (if uncomputed)
    if type(edge_cw) != torch.Tensor:
        edge_labels = y_edges.cpu().numpy().flatten()
        edge_cw = compute_class_weight("balanced", classes=np.unique(edge_labels), y=edge_labels)
    
    y_preds, loss = model(features[0].unsqueeze(0), features[1].unsqueeze(0), features[2].unsqueeze(0), features[3].unsqueeze(0))
    # net.forward(x_edges, x_edges_values, x_nodes, x_nodes_coord, y_edges, edge_cw)
    #logp = F.log_softmax(logits, 1)
    #loss = F.nll_loss(logp[mask], labels[mask])
    loss = loss.mean()
    

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset = EucTSPGraph()
# Dataset and attributes
graph = dataset[0]                               # Only 1 graph in this dataset
graph = graph.to(device)                      # Cast to GPU if available, else cpu
node_features = graph.ndata['weight']                 # [2708, 1433]: each node has a word vector of 1433 unique words
num_nodes = 50
node_adj = graph.ndata['adj']
node_coord = graph.ndata['node_coord']
node_labels = graph.ndata['label']                  # [2708]: each node has one label of range [0-6]
train_mask = graph.ndata['train_mask']
valid_mask = graph.ndata['val_mask']
test_mask = graph.ndata['test_mask']
num_feats = node_features.size()[1]
#num_classes = dataset.num_classes
labels = node_labels
features = [node_adj,node_features,node_labels,node_coord]
# GAT Hyperparameters
num_layers_graph = 30
num_layers_mlp = 3
hidden_dim = 300

model = ResidualGatedGCNModel(num_nodes, num_layers_graph, num_layers_mlp, hidden_dim)
model = model.to(device)

# create optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=5e-3, weight_decay=5e-4)

# Main
for epoch in range(300):

    loss = train(model, features, labels, train_mask)
    val_acc = evaluate(model, features, labels, valid_mask)

    # if epoch % 10 == 9:
    print("Epoch {:05d} | Loss {:.4f} | Accuracy {:.4f}".format(epoch+1, loss, val_acc))

# Testing
test_acc = evaluate(model, features, labels, test_mask)
print("Test Accuracy {:.4f}".format(test_acc))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


NameError: ignored